### Agent driven Auto Insurance Claims RAG Pipeline.

#### In this notebook, I'll demonstrate how to leverage Azure AI Document Intelligence in conjunction with AzureOpenAI's multimodal GPT-4o model to extract and interpret data from auto insurance claim documents that feature intricate tables. I'll utilize a template form that includes detailed sections on accident location, incident description, involved vehicles, and injury information.
#### I compare the responses to my sample questions against some ground truths, and so far, the answers are both accurate and relevant. Further refining the responses can be accomplished by applying additional preprocessing to the claims documents to better maintain context between the tables and associated texts. 
#### The claims files are in PDF format and contain tabular data.Azure AI Document Intelligence parses the table data into markdown-formatted tables, which can be chunked, indexed, uploaded to and queried over with a Azure AI Search index.
#### The goal of this proof of concept is to demonstrate how insurance companies can expedite the process of extracting information from car accident insurance claim documents. This can be achieved without the need to manually read through each claims form.
#### I set up autogen agents, incorporating a human-in-the-loop feature for automated chat interactions and responses.
#### Out of the 6 questions, 4 were accurate, one was incomplete, and the last could not be answered.
![Auto Insurance Claims Form in RAG](https://github.com/jbernec/rag-orchestrations/blob/main/images/claims-sample.jpeg?raw=true)

#### Step 1: Import required packages.

In [0]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest, ContentFormat
import time
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from autogen import AssistantAgent, UserProxyAgent, register_function
from typing_extensions import List, Annotated
import autogen
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchField, SearchFieldDataType, VectorSearch, SimpleField, SearchableField, HnswAlgorithmConfiguration, HnswParameters, VectorSearchAlgorithmMetric, ExhaustiveKnnAlgorithmConfiguration, ExhaustiveKnnParameters, VectorSearchProfile, AzureOpenAIVectorizer, AzureOpenAIParameters, SemanticConfiguration, SemanticSearch, SemanticPrioritizedFields, SemanticField, SearchIndex
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

#### Step 2: Set credential variables.

In [0]:
"""
This code loads and sets the necessary variables for Azure services.
The variables are loaded from Azure Key Vault.
"""

azure_openai_endpoint=dbutils.secrets.get(scope="myscope", key="aoai-endpoint")
azure_openai_api_key=dbutils.secrets.get(scope="myscope", key="aoai-api-key")
azure_openai_api_version = "2024-02-15-preview"
azure_openai_embedding_deployment = dbutils.secrets.get(scope="myscope", key="aoai-embedding-deployment")
doc_intelligence_endpoint = dbutils.secrets.get(scope="myscope", key="docintelligence-endpoint")
doc_intelligence_key = dbutils.secrets.get(scope="myscope", key="docintelligence-key")
search_credential = AzureKeyCredential(dbutils.secrets.get(scope="myscope", key="aisearch-adminkey"))
search_endpoint = dbutils.secrets.get(scope="myscope", key="aisearch-endpoint")

#### Step 3: Connect to blob storage.

In [0]:
# Connect to Blob Storage
blob_connection_string = dbutils.secrets.get(scope="myscope", key="blobstore-connstr")
blob_container_name = "insurance-rag"
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(blob_container_name)
blobs = container_client.list_blobs()
container_url = container_client.url
#print(container_url)

#### Step 4: Define and Configure Autogen Agents.

In [0]:
llm_config = {
    "config_list": [
        {
            "model": dbutils.secrets.get(scope="myscope", key="aoai-deploymentname"),
            "api_key": dbutils.secrets.get(scope="myscope", key="aoai-api-key"),
            "base_url": dbutils.secrets.get(scope="myscope", key="aoai-endpoint"),
            "api_type": "azure",
            "api_version": "2024-02-15-preview",
        },
    ]
}

gpt4_config = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": llm_config["config_list"],
    "timeout": 120
}


ai_search_agent = AssistantAgent(
    name="AISearchAssistant",
    system_message="You are a helpful AI agent."
    "You can help with Azure AI Search service."
    "Return TERMINATE when the task is done",
    llm_config=gpt4_config,
)

user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: "terminate" in x.get("content", "").lower()
    if x.get("content", "") is not None
    else False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=False,
)

#### Step 5: Define required functions and tools.

In [0]:
import base64

# Function to convert text to unique random id for search index field
def text_to_base64(text):
    # Convert text to bytes using UTF-8 encoding
    bytes_data = text.encode('utf-8')

    # Perform Base64 encoding
    base64_encoded = base64.b64encode(bytes_data)

    # Convert the result back to a UTF-8 string representation
    base64_text = base64_encoded.decode('utf-8')

    return base64_text




# Function to crack and extract PDF documents using Azure AI Document Intelligence
def extract_pdf_content(book_url: str):
    page_documents = ""
    print(f"{book_url}\n\n")
    print(f"---------------------------------------------")
    
    document_intelligence_client = DocumentIntelligenceClient(endpoint=doc_intelligence_endpoint, credential=AzureKeyCredential(key=doc_intelligence_key))

    poller= document_intelligence_client.begin_analyze_document(model_id="prebuilt-layout", analyze_request=AnalyzeDocumentRequest(url_source=book_url), output_content_format="markdown")

    result: AnalyzeResult = poller.result()
    
    for page in result.pages:
        page_num = page.page_number
        # Calculate the start position as the offset of the first span
        start_pos = page.spans[0].offset

        # Calculate the end position by adding the length of the first span to its offset
        end_pos = start_pos + page.spans[0].length

        # Slice the result.content string from start_pos to end_pos to get the desired content
        page_content = result.content[start_pos:end_pos]
        #print(f"{page_content}\n\n")

        #print(f"------------------------------------------")
        page_documents+=page_content

    
    return page_documents

# Define chunk strategy function
def chunk_text(text: str):
    pass
    recursive_text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name=dbutils.secrets.get(scope="myscope", key="aoai-deploymentname"),
        chunk_size=600,
        chunk_overlap=125,
        separators=["\n\n", "\n", " ", ""]
    )

    recursive_text_splitter_chunks = recursive_text_splitter.split_text(text=text)
    return recursive_text_splitter_chunks

# one way of registering functions is to use the register_for_llm and register_for_execution decorators or use the register_function method.

@user_proxy.register_for_execution()
@ai_search_agent.register_for_llm(
    description="A tool or function for search retrieval from Azure AI Search"
)
def search_retrieval(user_input:str) -> str:
        """
        Search and retrieve answers from Azure AI Search.
        Returns:
            str
        """
        query = user_input
        search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=search_credential)
        vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="vector", exhaustive=True)

        r = search_client.search(  
        search_text=query,
        vector_queries=[vector_query],
        select=["id", "content"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='my-semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=1
    )
        #query_result = results.get_answers()[0].text
        results = [doc["content"].replace("\n", "").replace("\r", "") for doc in r]
        content = "\n".join(results)
        return content

#### Step 6: Create Azure AI Search Index and Vector Configurations.

In [0]:
# Create the search index fields and vector search configuration

# Create a search index client required to create the index
index_client = SearchIndexClient(endpoint=search_endpoint, credential=search_credential)

fields = [
    SimpleField(name="id", key=True, type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String, filterable=True, searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True, sortable=True, facetable=True, retrievable=True),
    SearchableField(name="location", type=SearchFieldDataType.String, searchable=True, filterable=True, retrievable=True),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, retrievable=True, hidden=False, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
]

# Configure the vector search config
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer="myOpenAI",  
        ),
    ],
    vectorizers=[  
        AzureOpenAIVectorizer(  
            name="myOpenAI",  
            kind="azureOpenAI",  
            azure_open_ai_parameters=AzureOpenAIParameters(  
                resource_uri=azure_openai_endpoint,  
                deployment_id=azure_openai_embedding_deployment,  
                api_key=azure_openai_api_key,  
            ),  
        ),  
    ]
)

# Configure semantic search on the index
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[
            SemanticField(field_name="content")
        ]
    )
)
# Create the semantic search config
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index
index_name = "insuracer-rag-index"
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index=index)
print(f"{result.name} created")

#### Step 7: Configure vector embeddings and extract document text, tables and images into markdown.

In [0]:
# Create the langchain azure open ai embedding object. This will be used to embed the vector field content
# https://python.langchain.com/v0.1/docs/integrations/vectorstores/azuresearch/#create-embeddings-and-vector-store-instances

# Create azure open ai embedding
azure_openai_client = None
if azure_openai_api_key:
    azure_openai_client = AzureOpenAI(
        api_key=azure_openai_api_key, 
        api_version=azure_openai_api_version,
        azure_deployment=azure_openai_embedding_deployment,
        azure_endpoint=azure_openai_endpoint)
else:
    azure_openai_client = AzureOpenAI(
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), scope="https://cognitiveservices.azure.com/.default"),
        api_version=azure_openai_api_version,
        azure_deployment=azure_openai_embedding_deployment,
        azure_endpoint=azure_openai_endpoint)
    

aoai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
)

# dictionary to hold and map a book to it's content and page numbers
claims_pages_map = {}

for claim in container_client.list_blob_names():
    print(f"Extracting content from {claim}...")

    # Capture the start time
    start_time = time.time()
    book_url = container_url + "/" + claim

    # Start extraction
    page_documents = extract_pdf_content(book_url=book_url)
    claim_name = claim.split(sep=".")[0].title()
    chunks = chunk_text(page_documents)
    #chunked_docs = [Document(page_content=chunk) for chunk in chunks]
    claims_pages_map[claim_name]= chunks

    # Capture the end time and Calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Parsing took: {elapsed_time:.6f} seconds")
    print(f"The {claim_name} book contains {len(chunks)} chunks\n")

#### Step 8: Upload documents into Azure AI Search Index.

In [0]:
from azure.search.documents import SearchClient

search_client = SearchClient(search_endpoint, index_name, credential=search_credential)

for claimname, chunks in claims_pages_map.items():
    for chunk in chunks:
        try:
            id = claimname + chunk[1:10]
            title = f"{claimname}"
            upload_payload = {
                        "id": text_to_base64(text=id),
                        "title": title,
                        "content": chunk,
                        "location": container_url + "/" + claimname + ".pdf",
                        "vector": aoai_embeddings.embed_query(chunk if chunk!="" else "-------")
            }

            result_upload = search_client.upload_documents(documents=[upload_payload])
            #print(f"Successfully uploaded chunk for :{bookname}")
        except Exception as e:
            print("Exception:", e)

#### Step 9: Initiate Agent based Chat.

In [0]:
message = "Search for 'How did Ms. Patel's accident happen' in the above defined index?"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message)

In [0]:
message_2 = "Search for 'Who filed the insurance claim for the accident that happened on Sunset Blvd?' in the above defined index"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message_2)

In [0]:
message_3 = "Search for 'Given the accident that happened on Lombard Street, name a party that is liable for the damages and explain why.' in the above defined index"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message_3)

In [0]:
message_4 = "Search for 'Did Ms. Johnson sustain any injuries? If so, what were those injuries?' in the above defined index"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message_4)

In [0]:
message_5 = "Search for 'Who are some witnesses for the Ms. Patel's accident and how can we contact them?' in the above defined index"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message_5)

In [0]:
message_6 = "Search for 'How was Mr. Johnson's red sedan damaged?Give me details? What's the repair cost?' in the above defined index"

agent_response = await user_proxy.a_initiate_chat(recipient=ai_search_agent, message=message_6)